# Save and load forecasters

Skforecast models can be easily saved and loaded from disk using the pickle or joblib library. Two handy functions, `save_forecaster` and `load_forecaster` are available to streamline this process. See below for a simple example.

A `forecaster_id` has been included when initializing the Forecaster, this may help to identify the target of the model.

<div class="admonition note" name="html-admonition" style="background: rgba(0,184,212,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #00b8d4; border-color: #00b8d4; padding-left: 10px; padding-right: 10px;">

<p class="title">
    <i style="font-size: 18px; color:#00b8d4;"></i>
    <b style="color: #00b8d4;">&#9998 Note</b>
</p>

Learn how to use <a href="https://skforecast.org/latest/user_guides/forecaster-in-production">forecaster models in production</a>.

</div>

## Libraries

In [1]:
# Libraries
# ==============================================================================
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from skforecast.recursive import ForecasterRecursive
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster
from skforecast.datasets import fetch_dataset

## Data

In [2]:
# Download data
# ==============================================================================
data = fetch_dataset(
    name="h2o", raw=True, kwargs_read_csv={"names": ["y", "date"], "header": 0}
)
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data = data.set_index('date')
data = data.asfreq('MS')

h2o
---
Monthly expenditure ($AUD) on corticosteroid drugs that the Australian health
system had between 1991 and 2008.
Hyndman R (2023). fpp3: Data for Forecasting: Principles and Practice(3rd
Edition). http://pkg.robjhyndman.com/fpp3package/,https://github.com/robjhyndman
/fpp3package, http://OTexts.com/fpp3.
Shape of the dataset: (204, 2)


## Save and load forecaster model

In [3]:
# Create and train forecaster
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor     = RandomForestRegressor(random_state=123),
                 lags          = 5,
                 forecaster_id = "forecaster_001"
             )

forecaster.fit(y=data['y'])
forecaster.predict(steps=3)

2008-07-01    0.714526
2008-08-01    0.789144
2008-09-01    0.818433
Freq: MS, Name: pred, dtype: float64

In [4]:
# Save model
# ==============================================================================
save_forecaster(forecaster, file_name='forecaster_001.joblib', verbose=False)

In [5]:
# Load model
# ==============================================================================
forecaster_loaded = load_forecaster('forecaster_001.joblib', verbose=True)

ForecasterAutoreg 
Regressor: RandomForestRegressor(random_state=123) 
Lags: [1 2 3 4 5] 
Transformer for y: None 
Transformer for exog: None 
Window size: 5 
Weight function included: False 
Differentiation order: None 
Exogenous included: False 
Exogenous variables names: None 
Training range: [Timestamp('1991-07-01 00:00:00'), Timestamp('2008-06-01 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: MS 
Regressor parameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 123, 'verbose': 0, 'warm_start': False} 
fit_kwargs: {} 
Creation date: 2024-07-29 18:14:22 
Last fit date: 2024-07-29 18:14:22 
Skforecast version: 0.13.0 
Python version: 3.11.5 
Forec

In [6]:
# Predict
# ==============================================================================
forecaster_loaded.predict(steps=3)

2008-07-01    0.714526
2008-08-01    0.789144
2008-09-01    0.818433
Freq: MS, Name: pred, dtype: float64

In [7]:
# Forecaster identifier
# ==============================================================================
forecaster.forecaster_id

'forecaster_001'

## Saving and Loading a Forecaster Model with Custom Functions

Sometimes external functions are needed when creating a Forecaster object. For example:

+ A custom function to [create window and custom features](https://skforecast.org/latest/user_guides/window-features-and-custom-features).

+ A function to reduce the impact of some dates on the model, [Weighted Time Series Forecasting](https://skforecast.org/latest/user_guides/weighted-time-series-forecasting). 

For your code to work properly, these functions must be available in the environment where the Forecaster is loaded.

In [8]:
# Custom function to create predictors
# ==============================================================================
def create_predictors(y):
    """
    Create first 5 lags of a time series.
    """
    
    lags = y[-1:-6:-1]
    
    return lags

In [9]:
# Create and train forecaster
# ==============================================================================
forecaster = ForecasterAutoregCustom(
                 regressor      = RandomForestRegressor(random_state=123),
                 fun_predictors = create_predictors,
                 window_size    = 5
             )

forecaster.fit(y=data['y'])

The `save_forecaster` function will save all the functions used to create it in different files. In this case, the created module is `create_predictors.py`.

In [10]:
# Save model and custom function
# ==============================================================================
save_forecaster(
    forecaster, 
    file_name = 'forecaster_custom.joblib', 
    save_custom_functions = True, 
    verbose = False
)

These functions must be imported into the environment where the Forecaster is going to be loaded.

In [11]:
# Load model and custom function
# ==============================================================================
from create_predictors import create_predictors

forecaster_loaded = load_forecaster('forecaster_custom.joblib', verbose=True)

ForecasterAutoregCustom 
Regressor: RandomForestRegressor(random_state=123) 
Predictors created with function: create_predictors 
Transformer for y: None 
Transformer for exog: None 
Window size: 5 
Weight function included: False 
Differentiation order: None 
Exogenous included: False 
Type of exogenous variable: None 
Exogenous variables names: None 
Training range: [Timestamp('1991-07-01 00:00:00'), Timestamp('2008-06-01 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: MS 
Regressor parameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 123, 'verbose': 0, 'warm_start': False} 
fit_kwargs: {} 
Creation date: 2024-07-29 18:14:23 
Last fit date: 2024